## 1. Bluetooth

### 1) Bluetooth + PHQ-9
co-location + depression

pearson correlation 

phq post만 significant (r = 0.363, p = 0.025)

In [1]:
import csv
import glob
import pandas as pd
import numpy as np
import re
import time

In [2]:
# import bluetooth data
bt_path = '//data/dopameter/student_life/dataset/dataset/sensing/bluetooth/*.csv'
bt_data = {}
for file_name in glob.glob(bt_path):
    f = open(file_name, 'r')
    uid = int(re.search(r'\d+', file_name).group())  # user id
    bt_data[uid] = pd.read_csv(f)  # dict {uid: bluetooth dataframe}

In [3]:
# get avg number of bluetooth devices 

# execution time
# start_time = time.time()

# get total number of devices per each time period
bt_info = []
# bt_arr = []
# curr_time = bt_data[0].loc[0]['time'] - 1  # current time
# for index, row in bt_data[0].iterrows():
#     if row['time'] == curr_time:
#         bt_arr[-1][1] += 1
#     else:
#         curr_time = row['time']
#         bt_arr += [[curr_time, 1]]  # [time, num_devices]
# bt_df = pd.DataFrame(bt_arr, columns = ['time', 'num_devices'])
# # print(bt_df)

# # get mean
# bt_info += [0, round(bt_df['num_devices'].mean(), 3)]

# print(bt_info)

for key, value in bt_data.items():
    bt_arr = []
    curr_time = value.loc[0]['time'] - 1  # current time
    for i in value.index:
        row_time = value.at[i, 'time']
        row_level = value.at[i, 'level']
        if row_time == curr_time:
            if row_level > -90:
                bt_arr[-1][1] += 1
        else:
            curr_time = row_time
            if row_level > -90:
                bt_arr += [[curr_time, 1]]  # [time, num_devices]
            else:
                bt_arr += [[curr_time, 0]]
    bt_df = pd.DataFrame(bt_arr, columns = ['time', 'num_devices'])
    # get mean
    bt_info += [[key, round(bt_df['num_devices'].mean(), 3)]]

            
# print("%s seconds" % round(time.time() - start_time, 2))

In [4]:
print(bt_info)

[[42, 2.63], [59, 2.278], [18, 7.515], [30, 3.584], [4, 1.714], [34, 3.227], [49, 3.801], [51, 5.18], [22, 1.355], [0, 3.738], [2, 2.653], [45, 4.464], [36, 3.215], [13, 3.991], [9, 2.681], [41, 5.186], [14, 7.964], [1, 1.851], [17, 1.994], [46, 2.766], [39, 1.645], [3, 1.011], [35, 1.656], [19, 4.992], [24, 2.265], [56, 1.526], [47, 5.662], [52, 1.342], [27, 3.298], [20, 1.379], [50, 6.793], [25, 7.763], [15, 2.976], [10, 5.321], [53, 1.923], [44, 5.696], [32, 2.452], [12, 2.96], [43, 3.121], [16, 4.294], [5, 6.495], [33, 1.537], [58, 2.572], [57, 2.363], [31, 5.771], [23, 1.505], [54, 3.553], [8, 3.672], [7, 1.535]]


In [5]:
bt = pd.DataFrame(bt_info, columns = ['uid', 'avg_bt'])
bt = bt.sort_values(by='uid').reset_index(drop=True)
bt.head()

,uid,avg_bt
0,0,3.738
1,1,1.851
2,2,2.653
3,3,1.011
4,4,1.714


In [6]:
# import phq-9 data
phq_path = '//data/dopameter/student_life/dataset/dataset/survey/PHQ-9.csv'
f = open(phq_path, 'r')
phq_df = pd.read_csv(f)

In [7]:
# get scores
phq_scores = {}  # {uid: uid, phq-9_pre, phq-9_post}

# pre
for index, row in phq_df.loc[phq_df['type'] == 'pre'].iterrows():
    score = 0
    for i in range(2, 11):
        if row[i] == 'Not at all':
            score += 0
        elif row[i] == 'Several days':
            score += 1
        elif row[i] == 'More than half the days':
            score += 2
        elif row[i] == 'Nearly every day':
            score += 3
        else:
            print("errorrrr")
    uid = int(re.search(r'\d+', row['uid']).group())
    phq_scores[uid] = [uid, score]
    
# post
for index, row in phq_df.loc[phq_df['type'] == 'post'].iterrows():
    score = 0
    for i in range(2, 11):
        if row[i] == 'Not at all':
            score += 0
        elif row[i] == 'Several days':
            score += 1
        elif row[i] == 'More than half the days':
            score += 2
        elif row[i] == 'Nearly every day':
            score += 3
        else:
            print("errorrrr")
    uid = int(re.search(r'\d+', row['uid']).group())
    phq_scores[uid] += [score]
    
phq = pd.DataFrame.from_dict(phq_scores, columns = ['uid', 'phq-9_pre', 'phq-9_post'], orient='index')

In [8]:
phq = phq.sort_values(by='uid').reset_index(drop=True)
phq.head()

,uid,phq-9_pre,phq-9_post
0,0,2,3.0
1,1,5,4.0
2,2,13,5.0
3,3,2,4.0
4,4,6,8.0


In [9]:
# merge bt & phq 
bt_phq = pd.merge(bt, phq, how='outer').sort_values(by='uid').reset_index(drop=True)
bt_phq['uid'] = bt_phq['uid'].astype(int)
bt_phq.head()

,uid,avg_bt,phq-9_pre,phq-9_post
0,0,3.738,2.0,3.0
1,1,1.851,5.0,4.0
2,2,2.653,13.0,5.0
3,3,1.011,2.0,4.0
4,4,1.714,6.0,8.0


In [10]:
import scipy.stats

In [11]:
# drop nan
bt_phq_valid = bt_phq.dropna()

In [12]:
r, p = scipy.stats.pearsonr(bt_phq_valid['avg_bt'], bt_phq_valid['phq-9_post'])

In [13]:
r, p  # same as original study !! (when bluetooth threshold = -90)

(-0.3629788425645571, 0.025103091650760235)

In [14]:
bt_phq['phq-9_avg'] =bt_phq[['phq-9_pre', 'phq-9_post']].mean(axis=1)

In [15]:
bt_phq.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg
0,0,3.738,2.0,3.0,2.5
1,1,1.851,5.0,4.0,4.5
2,2,2.653,13.0,5.0,9.0
3,3,1.011,2.0,4.0,3.0
4,4,1.714,6.0,8.0,7.0


In [16]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_bt', 'phq-9_post', -0.363, 0.025]]
sig_corr = pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p'])
sig_corr.to_csv('sig_corr.csv')

### 2) Bluetooth + Flourising
co-location + flourising

pearson correlation

not significant

In [17]:
# import flourishing data
fl_path = '//data/dopameter/student_life/dataset/dataset/survey/FlourishingScale.csv'
f = open(fl_path, 'r')
fl_df = pd.read_csv(f)


In [18]:
# get scores
fl_scores = {}  # {uid: uid, fl_pre, fl_post}

# pre
for index, row in fl_df.loc[fl_df['type'] == 'pre'].iterrows():
    score = 0
    for i in range(2, 10):
        score += row[i]
    uid = int(re.search(r'\d+', row['uid']).group())
    fl_scores[uid] = [uid, score]
    
# post
for index, row in fl_df.loc[fl_df['type'] == 'post'].iterrows():
    score = 0
    for i in range(2, 10):
        score += row[i]
    uid = int(re.search(r'\d+', row['uid']).group())
    fl_scores[uid] += [score]
    
fl = pd.DataFrame.from_dict(fl_scores, columns = ['uid', 'fl_pre', 'fl_post'], orient='index')



In [19]:
fl = fl.sort_values(by='uid').reset_index(drop=True)
fl.head()

,uid,fl_pre,fl_post
0,0,NaN,45.0
1,1,45.0,46.0
2,2,46.0,44.0
3,3,34.0,31.0
4,4,27.0,33.0


In [20]:
# merge bt_phq & fl 
bt = pd.merge(bt_phq, fl, how='outer').sort_values(by='uid').reset_index(drop=True)
bt['uid'] = bt['uid'].astype(int)
bt.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post
0,0,3.738,2.0,3.0,2.5,NaN,45.0
1,1,1.851,5.0,4.0,4.5,45.0,46.0
2,2,2.653,13.0,5.0,9.0,46.0,44.0
3,3,1.011,2.0,4.0,3.0,34.0,31.0
4,4,1.714,6.0,8.0,7.0,27.0,33.0


In [21]:
bt_valid = bt.dropna()

In [22]:
r, p = scipy.stats.pearsonr(bt_valid['avg_bt'], bt_valid['fl_post'])

In [23]:
r, p  # not significant

(0.19833488925732234, 0.26853076723431235)

In [24]:
r, p = scipy.stats.pearsonr(bt_valid['avg_bt'], bt_valid['fl_pre'])
r, p

(0.12369335987604126, 0.49283482928771843)

### 3) Bluetooth + Big Five
co-location + personality

pearson correlation

Agreeableness (pre) 만 significant (r = 0.324, p = 0.047)

In [25]:
# import Big Five data
big5_path = '//data/dopameter/student_life/dataset/dataset/survey/BigFive.csv'
f = open(big5_path, 'r')
big5_df = pd.read_csv(f)

In [26]:
# get scores
big5_scores = {}  # {uid: uid, big5_pre, big5_post}

reverse = [2, 6, 8, 9, 12, 18, 21, 23, 24, 27, 31, 34, 35, 37, 41, 43]
# pre
pre = big5_df.loc[big5_df['type'] == 'pre']
for index in pre.index:
    o, c, e, a, n = 0, 0, 0, 0, 0
    for i in range(2, 46):
        response = pre.iat[index, i]
        to_add = 0
        if response == 'Disagree Strongly':
            to_add = 1
        elif response == 'Disagree a little':
            to_add = 2
        elif response == 'Neither agree nor disagree':
            to_add = 3
        elif response == 'Agree a little':
            to_add = 4
        elif response == 'Agree strongly':
            to_add = 5
        else:
            print(response)
            to_add = 3  # add average value for missing data

        if i-1 in reverse:
            to_add = 6 - to_add
            
        if i == 42 or i == 45 or i % 5 == 1:
            o += to_add
        elif i % 5 == 2:
            e += to_add
        elif i % 5 == 3:
            a += to_add
        elif i % 5 == 4:
            c += to_add
        elif i % 5 == 0:
            n += to_add

    uid = int(re.search(r'\d+', pre.at[index, 'uid']).group())
    big5_scores[uid] = [uid, o, c, e, a, n]

# post
post = big5_df.loc[big5_df['type'] == 'post'].reset_index(drop=True)
for index in post.index:
    o, c, e, a, n = 0, 0, 0, 0, 0
    for i in range(2, 46):
        response = post.iloc[index, i]
        to_add = 0
        if response == 'Disagree Strongly':
            to_add = 1
        elif response == 'Disagree a little':
            to_add = 2
        elif response == 'Neither agree nor disagree':
            to_add = 3
        elif response == 'Agree a little':
            to_add = 4
        elif response == 'Agree strongly':
            to_add = 5
        else:
            print(response)
            to_add = 3  # add average value for missing data

        if i-1 in reverse:
            to_add = 6 - to_add
            
        if i == 42 or i == 45 or i % 5 == 1:
            o += to_add
        elif i % 5 == 2:
            e += to_add
        elif i % 5 == 3:
            a += to_add
        elif i % 5 == 4:
            c += to_add
        elif i % 5 == 0:
            n += to_add

    uid = int(re.search(r'\d+', post.at[index, 'uid']).group())
    if uid in big5_scores:
        big5_scores[uid] += [o, c, e, a, n]
    else:
        big5_scores[uid] = [uid, None, None, None, None, None, o, c, e, a, n]
    
    
big5 = pd.DataFrame.from_dict(big5_scores, columns = ['uid', 
                                                      'big5-o_pre', 'big5-c_pre', 
                                                      'big5-e_pre', 'big5-a_pre', 
                                                      'big5-n_pre', 'big5-o_post', 
                                                      'big5-c_post', 'big5-e_post', 
                                                      'big5-a_post', 'big5-n_post'
                                                     ], 
                             orient='index'
                             )

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [27]:
big5.head()

,uid,big5-o_pre,big5-c_pre,big5-e_pre,big5-a_pre,big5-n_pre,big5-o_post,big5-c_post,big5-e_post,big5-a_post,big5-n_post
0,0,45.0,27.0,30.0,26.0,25.0,46.0,26.0,28.0,32.0,24.0
1,1,39.0,40.0,25.0,34.0,22.0,37.0,38.0,24.0,34.0,23.0
2,2,32.0,38.0,17.0,27.0,28.0,30.0,35.0,15.0,27.0,26.0
3,3,27.0,35.0,17.0,29.0,27.0,27.0,35.0,19.0,26.0,24.0
4,4,29.0,24.0,19.0,28.0,33.0,31.0,24.0,22.0,27.0,37.0


In [28]:
# merge bt & big5 
# bt = bt.drop(['big5-o_pre', 'big5-c_pre', 'big5-e_pre', 'big5-a_pre', 'big5-n_pre', 'big5-o_post', 'big5-c_post', 'big5-e_post', 'big5-a_post', 'big5-n_post'], axis=1)
bt = pd.merge(bt, big5, how='outer').sort_values(by='uid').reset_index(drop=True)
bt.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,big5-a_pre,big5-n_pre,big5-o_post,big5-c_post,big5-e_post,big5-a_post,big5-n_post
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,26.0,25.0,46.0,26.0,28.0,32.0,24.0
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,34.0,22.0,37.0,38.0,24.0,34.0,23.0
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,27.0,28.0,30.0,35.0,15.0,27.0,26.0
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,29.0,27.0,27.0,35.0,19.0,26.0,24.0
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,28.0,33.0,31.0,24.0,22.0,27.0,37.0


In [29]:
bt_big5_valid = bt[['uid', 'avg_bt', 'big5-o_pre', 'big5-c_pre', 'big5-e_pre', 'big5-a_pre', 'big5-n_pre', 'big5-o_post', 'big5-c_post', 'big5-e_post', 'big5-a_post', 'big5-n_post']].dropna()

In [30]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-o_pre'])

In [31]:
r, p  # not significant

(-0.11010865521581585, 0.5104822346066332)

In [32]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-c_pre'])

In [33]:
r, p  # not significant

(0.260066703239749, 0.11482418326616602)

In [34]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-e_pre'])

In [35]:
r, p  # not significant

(0.21315880431827158, 0.19881612076521585)

In [36]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-a_pre'])

In [37]:
r, p  # significant

(-0.32398157031574937, 0.04721958084543573)

In [38]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-n_pre'])

In [39]:
r, p  # not significant

(-0.09256713063022229, 0.58043494391738)

In [40]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-o_post'])

In [41]:
r, p  # significant

(-0.049820086111971376, 0.7664382486498298)

In [42]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-c_post'])

In [43]:
r, p  # not significant

(0.2708853608028628, 0.09997242656729413)

In [44]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-e_post'])

In [45]:
r, p  # not significant

(0.05299563249286259, 0.75200543074575)

In [46]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-a_post'])

In [47]:
r, p  # not significant

(-0.30733835452318137, 0.06051980513071529)

In [48]:
r, p = scipy.stats.pearsonr(bt_big5_valid['avg_bt'], bt_big5_valid['big5-n_post'])

In [49]:
r, p  # not significant

(-0.18270620554112071, 0.272236381407248)

In [50]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_bt', 'big5-a_pre', -0.324, 0.047]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

### 4) Bluetooth + Loneliness
co-location + loneliness

pearson correlation

not significant

In [51]:
# import Big Five data
path = '//data/dopameter/student_life/dataset/dataset/survey/LonelinessScale.csv'
f = open(path, 'r')
lon_df = pd.read_csv(f)

In [52]:
# get scores
lon_scores = {}  # {uid: uid, lon_pre, lon_post}

# pre
pre = lon_df.loc[lon_df['type'] == 'pre']
for index in pre.index:
    score = 0
    for i in range(2, 22):
        response = pre.iat[index, i]
        if response == 'Often':
            score += 3
        elif response == 'Sometimes':
            score += 2
        elif response == 'Rarely':
            score += 1
        elif response == 'Never':
            score += 0
        else:
            print(response)
            score += 1.5  # add average value for missing data

    uid = int(re.search(r'\d+', pre.at[index, 'uid']).group())
    lon_scores[uid] = [uid, score]

# post
post = lon_df.loc[lon_df['type'] == 'post'].reset_index(drop=True)
for index in post.index:
    score = 0
    for i in range(2, 22):
        response = post.iat[index, i]
        if response == 'Often':
            score += 3
        elif response == 'Sometimes':
            score += 2
        elif response == 'Rarely':
            score += 1
        elif response == 'Never':
            score += 0
        else:
            print(response)
            score += 1.5  # add average value for missing data

    uid = int(re.search(r'\d+', post.at[index, 'uid']).group())
    lon_scores[uid] += [score]
    
lon = pd.DataFrame.from_dict(lon_scores, columns = ['uid', 'lon_pre', 'lon_post'], orient='index')

In [53]:
lon.head()

,uid,lon_pre,lon_post
0,0,31,33.0
1,1,34,39.0
2,2,33,31.0
3,3,33,25.0
4,4,31,36.0


In [54]:
# merge bt & lon 
# bt = bt.drop(['lon_pre', 'lon_post'], axis=1)
bt = pd.merge(bt, lon, how='outer').sort_values(by='uid').reset_index(drop=True)
bt.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,big5-a_pre,big5-n_pre,big5-o_post,big5-c_post,big5-e_post,big5-a_post,big5-n_post,lon_pre,lon_post
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,26.0,25.0,46.0,26.0,28.0,32.0,24.0,31.0,33.0
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,34.0,22.0,37.0,38.0,24.0,34.0,23.0,34.0,39.0
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,27.0,28.0,30.0,35.0,15.0,27.0,26.0,33.0,31.0
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,29.0,27.0,27.0,35.0,19.0,26.0,24.0,33.0,25.0
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,28.0,33.0,31.0,24.0,22.0,27.0,37.0,31.0,36.0


In [55]:
bt_lon_valid = bt[['uid', 'avg_bt', 'lon_pre', 'lon_post']].dropna()

In [56]:
r, p = scipy.stats.pearsonr(bt_lon_valid['avg_bt'], bt_lon_valid['lon_pre'])

In [57]:
r, p  # significant

(-0.28771337782819695, 0.0842129083288769)

In [58]:
r, p = scipy.stats.pearsonr(bt_lon_valid['avg_bt'], bt_lon_valid['lon_post'])

In [59]:
r, p  # not significant

(-0.3121369307642529, 0.06000125882424857)

### 5) Bluetooth + PANAS
co-location + positive/negative affect

pearson correlation

not significant

In [60]:
# import PANAS data
path = '//data/dopameter/student_life/dataset/dataset/survey/panas.csv'
f = open(path, 'r')
panas_df = pd.read_csv(f)

In [61]:
# get scores
panas_scores = {}  # {uid: uid, pre, post}

pos_items = [1,4,8,9,11,12,14,15,17]
neg_items = [2,3,5,6,7,10,13,16,18]

# pre
pre = panas_df.loc[panas_df['type'] == 'pre']
for index in pre.index:
    pos, neg = 0, 0
    to_add = 0
    for i in range(2, 20):
        to_add = pre.iat[index, i]
        if pd.isna(to_add):
            to_add = 3  # add average value for missing data
            
        if i-1 in pos_items:
            pos += to_add
        elif i-1 in neg_items:
            neg += to_add
        else:
            print("errorrr")
    uid = int(re.search(r'\d+', pre.at[index, 'uid']).group())
    panas_scores[uid] = [uid, pos, neg]
    
# post
post = panas_df.loc[panas_df['type'] == 'post'].reset_index(drop=True)
for index in post.index:
    pos, neg = 0, 0
    to_add = 0
    for i in range(2, 20):
        to_add = post.iat[index, i]
        if pd.isna(to_add):
            to_add = 3  # add average value for missing data
            
        if i-1 in pos_items:
            pos += to_add
        elif i-1 in neg_items:
            neg += to_add
        else:
            print("errorrr")
    uid = int(re.search(r'\d+', post.at[index, 'uid']).group())
    if uid in panas_scores:
        panas_scores[uid] += [pos, neg]
    else:
        panas_scores[uid] = [uid, None, None, pos, neg]
    
panas = pd.DataFrame.from_dict(panas_scores, columns = ['uid', 'panas-pos_pre', 'panas-neg_pre', 'panas-pos_post', 'panas-neg_post'], orient='index')


In [62]:
panas.head()

,uid,panas-pos_pre,panas-neg_pre,panas-pos_post,panas-neg_post
0,0,32.0,33.0,31.0,24.0
1,1,30.0,19.0,27.0,17.0
2,2,23.0,15.0,28.0,13.0
3,3,30.0,27.0,25.0,20.0
4,4,27.0,14.0,34.0,19.0


In [63]:
# merge bt & lon 
# bt = bt.drop(['panas-pos_pre', 'panas-neg_pre', 'panas-pos_post', 'panas-neg_post'], axis=1)
bt = pd.merge(bt, panas, how='outer').sort_values(by='uid').reset_index(drop=True)
bt.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,...,big5-c_post,big5-e_post,big5-a_post,big5-n_post,lon_pre,lon_post,panas-pos_pre,panas-neg_pre,panas-pos_post,panas-neg_post
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,...,26.0,28.0,32.0,24.0,31.0,33.0,32.0,33.0,31.0,24.0
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,...,38.0,24.0,34.0,23.0,34.0,39.0,30.0,19.0,27.0,17.0
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,...,35.0,15.0,27.0,26.0,33.0,31.0,23.0,15.0,28.0,13.0
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,...,35.0,19.0,26.0,24.0,33.0,25.0,30.0,27.0,25.0,20.0
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,...,24.0,22.0,27.0,37.0,31.0,36.0,27.0,14.0,34.0,19.0


In [64]:
bt_panas_valid = bt[['uid', 'avg_bt', 'panas-pos_pre', 'panas-neg_pre', 'panas-pos_post', 'panas-neg_post']].dropna()

In [65]:
r, p = scipy.stats.pearsonr(bt_panas_valid['avg_bt'], bt_panas_valid['panas-pos_pre'])
r, p  # not significant

(0.0990742827113386, 0.5539897205228486)

In [66]:
r, p = scipy.stats.pearsonr(bt_panas_valid['avg_bt'], bt_panas_valid['panas-neg_pre'])
r, p  # significant

(0.28072331632448566, 0.08778348971330797)

In [67]:
r, p = scipy.stats.pearsonr(bt_panas_valid['avg_bt'], bt_panas_valid['panas-pos_post'])
r, p  # not significant

(0.10678036162436687, 0.5234227873598007)

In [68]:
r, p = scipy.stats.pearsonr(bt_panas_valid['avg_bt'], bt_panas_valid['panas-neg_post'])
r, p  # not significant

(-0.03796568759052274, 0.8209683502176397)

### 6) Bluetooth + Perceived Stress Scale
co-location + scale

pearson correlation

not significant

In [69]:
# import PSS data
path = '//data/dopameter/student_life/dataset/dataset/survey/PerceivedStressScale.csv'
f = open(path, 'r')
pss_df = pd.read_csv(f)

In [70]:
# get scores
pss_scores = {}  # {uid: uid, pss_pre, pss_post}

reverse = [4, 5, 7, 8]
# pre
pre = pss_df.loc[pss_df['type'] == 'pre']
for index in pre.index:
    score = 0
    for i in range(2, 12):
        to_add = 0
        response = pre.iat[index, i]
        if response == 'Very often':
            to_add = 4
        elif response == 'Fairly often':
            to_add = 3
        elif response == 'Sometime':
            to_add = 2
        elif response == 'Almost never':
            to_add = 1
        elif response == 'Never':
            to_add = 0
        else:
            print(response)
            to_add = 2  # add average value for missing data
            
        if i-1 in reverse:
            to_add = 4 - to_add
            
        score += to_add

    uid = int(re.search(r'\d+', pre.at[index, 'uid']).group())
    pss_scores[uid] = [uid, score]

# post
post = pss_df.loc[pss_df['type'] == 'post'].reset_index(drop=True)
for index in post.index:
    score = 0
    for i in range(2, 12):
        to_add = 0
        response = post.iat[index, i]
        if response == 'Very often':
            to_add = 4
        elif response == 'Fairly often':
            to_add = 3
        elif response == 'Sometime':
            to_add = 2
        elif response == 'Almost never':
            to_add = 1
        elif response == 'Never':
            to_add = 0
        else:
            print(response)
            to_add = 2  # add average value for missing data
            
        if i-1 in reverse:
            to_add = 4 - to_add
            
        score += to_add

    uid = int(re.search(r'\d+', post.at[index, 'uid']).group())
    pss_scores[uid] += [score]
    
pss = pd.DataFrame.from_dict(pss_scores, columns = ['uid', 'pss_pre', 'pss_post'], orient='index')

nan
nan


In [71]:
pss.head()

,uid,pss_pre,pss_post
0,0,24,15.0
1,1,15,16.0
2,2,21,25.0
3,3,17,24.0
4,4,21,17.0


In [72]:
# merge
# bt = bt.drop(['panas-pos_pre', 'panas-neg_pre', 'panas-pos_post', 'panas-neg_post'], axis=1)
bt = pd.merge(bt, pss, how='outer').sort_values(by='uid').reset_index(drop=True)
bt.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,...,big5-a_post,big5-n_post,lon_pre,lon_post,panas-pos_pre,panas-neg_pre,panas-pos_post,panas-neg_post,pss_pre,pss_post
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,...,32.0,24.0,31.0,33.0,32.0,33.0,31.0,24.0,24.0,15.0
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,...,34.0,23.0,34.0,39.0,30.0,19.0,27.0,17.0,15.0,16.0
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,...,27.0,26.0,33.0,31.0,23.0,15.0,28.0,13.0,21.0,25.0
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,...,26.0,24.0,33.0,25.0,30.0,27.0,25.0,20.0,17.0,24.0
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,...,27.0,37.0,31.0,36.0,27.0,14.0,34.0,19.0,21.0,17.0


In [73]:
bt_pss_valid = bt[['uid', 'avg_bt', 'pss_pre', 'pss_post']].dropna()

In [74]:
r, p = scipy.stats.pearsonr(bt_pss_valid['avg_bt'], bt_pss_valid['pss_pre'])
r, p  # not significant

(-0.1287485155447949, 0.4347237728729937)

In [75]:
r, p = scipy.stats.pearsonr(bt_pss_valid['avg_bt'], bt_pss_valid['pss_post'])
r, p  # not significant

(-0.1583441376174915, 0.3356588920726596)

### 7) Bluetooth + GPA
co-location + GPA

pearson correlation

spring term GPA만 significant (r = 0.242, p = 0.290)

In [76]:
# import gpa data
gpa_path = '//data/dopameter/student_life/dataset/dataset/education/grades.csv'
f = open(gpa_path, 'r')
gpa_df = pd.read_csv(f, names=['uid', 'gpa_all', 'gpa_13s', 'cs_65'], header=0)

In [77]:
# change uid to int
for index, row in gpa_df.iterrows():
    gpa_df.loc[index, 'uid'] = int(re.search(r'\d+', row['uid']).group())

gpa_df.head()

,uid,gpa_all,gpa_13s,cs_65
0,1,2.863,1.777778,3.0
1,2,3.505,4.000000,4.0
2,4,3.029,3.500000,4.0
3,5,3.679,3.777778,4.0
4,7,3.474,4.000000,4.0


In [78]:
# merge bt_phq & gpa 
bt = pd.merge(bt, gpa_df, how='outer').sort_values(by='uid').reset_index(drop=True)
bt['uid'] = bt['uid'].astype(int)
bt.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,...,lon_post,panas-pos_pre,panas-neg_pre,panas-pos_post,panas-neg_post,pss_pre,pss_post,gpa_all,gpa_13s,cs_65
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,...,33.0,32.0,33.0,31.0,24.0,24.0,15.0,NaN,NaN,NaN
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,...,39.0,30.0,19.0,27.0,17.0,15.0,16.0,2.863,1.777778,3.0
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,...,31.0,23.0,15.0,28.0,13.0,21.0,25.0,3.505,4.000000,4.0
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,...,25.0,30.0,27.0,25.0,20.0,17.0,24.0,NaN,NaN,NaN
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,...,36.0,27.0,14.0,34.0,19.0,21.0,17.0,3.029,3.500000,4.0


In [79]:
bt_valid = bt.dropna()

In [80]:
r, p = scipy.stats.pearsonr(bt_valid['avg_bt'], bt_valid['gpa_all'])

In [81]:
r, p  # not significant

(0.3771585531965868, 0.10115176064324082)

In [82]:
r, p = scipy.stats.pearsonr(bt_valid['avg_bt'], bt_valid['gpa_13s'])

In [83]:
r, p  # significant (when bluetooth threshold = -90)

(0.4467736010431869, 0.0482844550172856)

In [84]:
r, p = scipy.stats.pearsonr(bt_valid['avg_bt'], bt_valid['cs_65'])

In [85]:
r, p

(0.21889587112695916, 0.35381576639725704)

In [86]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_bt', 'gpa_13s', 0.447, 0.048]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

## 2. GPS
### 1) distance traveled across the day

In [87]:
# import gps data
gps_path = '//data/dopameter/student_life/dataset/dataset/sensing/gps/*.csv'
gps_data = {}
for file_name in glob.glob(gps_path):
    f = open(file_name, 'r')
    uid = int(re.search(r'\d+', file_name).group())  # user id
    gps_data[uid] = pd.read_csv(f, index_col=False)  # dict {uid: gps dataframe}

In [88]:
import datetime

def convert_datetime(unixtime):
    """Convert unixtime to datetime"""
    date = datetime.datetime.fromtimestamp(unixtime)
    return date

In [89]:
import math

def calculate_distance(p1, p2):
    return math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)+((p1[2]-p2[2])**2) )

In [90]:
# get avg number traveled distance per day

import scipy

# curr_date = None
# days_distance = {}
# prev_location = [None, None, None]  # [lat, long, alt]
# for i in gps_data[0].index:
#     row_datetime = convert_datetime(gps_data[0].at[i, 'time'])
#     row_lat = gps_data[0].at[i, 'latitude']
#     row_long = gps_data[0].at[i, 'longitude']
#     row_alt = gps_data[0].at[i, 'altitude']
#     row_location = [row_lat, row_long, row_alt]
# #     print(row_datetime.date().strftime('%Y-%m-%d'))
          
#     if row_datetime.date() == curr_date:
#         days_distance[curr_date] += calculate_distance(prev_location, row_location)
#     else:
#         curr_date = row_datetime.date()
#         days_distance[curr_date] = 0
#     prev_location = [row_lat, row_long, row_alt]

# for date in days_distance:
#     days_distance[date] = round(days_distance[date], 3)

user_distance = {}  # {uid: day distance df}
distance_info = {} # {uid: avg distance per day}
# get total distance for each day
for key, value in gps_data.items():
    curr_date = None
    days_distance = {}
    prev_location = [None, None, None]  # [lat, long, alt]
    for i in value.index:
        row_datetime = convert_datetime(value.at[i, 'time'])
        row_lat = value.at[i, 'latitude']
        row_long = value.at[i, 'longitude']
        row_alt = value.at[i, 'altitude']
        row_location = [row_lat, row_long, row_alt]
    #     print(row_datetime.date().strftime('%Y-%m-%d'))

        if row_datetime.date() == curr_date:
            days_distance[curr_date] += calculate_distance(prev_location, row_location)
        else:
            curr_date = row_datetime.date()
            days_distance[curr_date] = 0
        prev_location = [row_lat, row_long, row_alt]

    for date in days_distance:
        days_distance[date] = round(days_distance[date], 3)
    
    days_distance_df = pd.DataFrame.from_dict(days_distance, columns = ['distance'], orient='index')

    user_distance[key] = days_distance_df
    
    # get mean distance per day
    distance_info[key] = [round(days_distance_df['distance'].mean(), 3)]

distance = pd.DataFrame.from_dict(distance_info, columns = ['avg_dist'], orient='index')


In [91]:
distance = distance.reset_index().rename(columns={'index': 'uid'})
distance = distance.sort_values(by='uid').reset_index(drop=True)

In [92]:
distance.head()

,uid,avg_dist
0,0,808.349
1,1,1505.468
2,2,852.401
3,3,486.413
4,4,324.165


#### 1.1) travel distance + phq
not significant

In [93]:
df = bt

In [94]:
# merge distance & rest data(df) 
df = pd.merge(df, distance, how='outer').sort_values(by='uid').reset_index(drop=True)
df.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,...,panas-pos_pre,panas-neg_pre,panas-pos_post,panas-neg_post,pss_pre,pss_post,gpa_all,gpa_13s,cs_65,avg_dist
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,...,32.0,33.0,31.0,24.0,24.0,15.0,NaN,NaN,NaN,808.349
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,...,30.0,19.0,27.0,17.0,15.0,16.0,2.863,1.777778,3.0,1505.468
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,...,23.0,15.0,28.0,13.0,21.0,25.0,3.505,4.000000,4.0,852.401
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,...,30.0,27.0,25.0,20.0,17.0,24.0,NaN,NaN,NaN,486.413
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,...,27.0,14.0,34.0,19.0,21.0,17.0,3.029,3.500000,4.0,324.165


In [95]:
dist_phq_valid = df[['uid', 'phq-9_pre', 'phq-9_post', 'avg_dist']].dropna()

In [96]:
r, p = scipy.stats.pearsonr(dist_phq_valid['avg_dist'], dist_phq_valid['phq-9_pre'])

In [97]:
r, p  # not significant

(0.18690340540897224, 0.26118338584346895)

In [98]:
r, p = scipy.stats.pearsonr(dist_phq_valid['avg_dist'], dist_phq_valid['phq-9_post'])

In [99]:
r, p  # not significant

(0.25596673836067985, 0.12086699622601683)

#### 1.2) travel distance + flourishing
not significant

In [100]:
dist_fl_valid = df[['uid', 'fl_pre', 'fl_post', 'avg_dist']].dropna()

In [101]:
r, p = scipy.stats.pearsonr(dist_fl_valid['avg_dist'], dist_fl_valid['fl_pre'])

In [102]:
r, p  # not significant

(0.009968018819220317, 0.9553814141300503)

In [103]:
r, p = scipy.stats.pearsonr(dist_fl_valid['avg_dist'], dist_fl_valid['fl_post'])

In [104]:
r, p  # not significant

(-0.00029740507624985386, 0.9986681023228025)

#### 1.3) travel distance + loneliness
not significant

In [105]:
dist_lon_valid = df[['uid', 'lon_pre', 'lon_post', 'avg_dist']].dropna()

In [106]:
r, p = scipy.stats.pearsonr(dist_lon_valid['avg_dist'], dist_lon_valid['lon_pre'])
r, p

(0.15041390881683792, 0.37421819047735805)

In [107]:
r, p = scipy.stats.pearsonr(dist_lon_valid['avg_dist'], dist_lon_valid['lon_post'])
r, p

(-0.10697465810834877, 0.528576742797115)

#### 1.4) travel distance + PANAS
not significant

In [108]:
dist_panas_valid = df[['uid', 'panas-pos_pre', 'panas-neg_pre', 'panas-pos_post', 'panas-neg_post', 'avg_dist']].dropna()

In [109]:
r, p = scipy.stats.pearsonr(dist_panas_valid['avg_dist'], dist_panas_valid['panas-pos_pre'])
r, p

(0.15233463097629046, 0.3612272824354882)

In [110]:
r, p = scipy.stats.pearsonr(dist_panas_valid['avg_dist'], dist_panas_valid['panas-neg_pre'])
r, p

(0.10915822199166697, 0.5141611469406951)

In [111]:
r, p = scipy.stats.pearsonr(dist_panas_valid['avg_dist'], dist_panas_valid['panas-pos_post'])
r, p

(-0.1085267094993062, 0.516612868923303)

In [112]:
r, p = scipy.stats.pearsonr(dist_panas_valid['avg_dist'], dist_panas_valid['panas-neg_post'])
r, p

(0.07669002515768457, 0.6472169759365486)

#### 1.5) travel distance + stress
not significant

In [113]:
dist_pss_valid = df[['uid', 'pss_pre', 'pss_post', 'avg_dist']].dropna()

In [114]:
r, p = scipy.stats.pearsonr(dist_pss_valid['avg_dist'], dist_pss_valid['pss_pre'])
r, p

(0.17464816982410478, 0.28760454249881445)

In [115]:
r, p = scipy.stats.pearsonr(dist_pss_valid['avg_dist'], dist_pss_valid['pss_post'])
r, p

(0.07176297987394019, 0.6641874423248492)

#### 1.6) travel distance + big five
not significant

In [116]:
dist_big5_valid = df[['uid', 'big5-o_pre', 'big5-c_pre', 
                              'big5-e_pre', 'big5-a_pre', 
                              'big5-n_pre', 'big5-o_post', 
                              'big5-c_post', 'big5-e_post', 
                              'big5-a_post', 'big5-n_post', 'avg_dist']].dropna()

In [117]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-o_pre'])
r, p

(0.07625183263723082, 0.6491025323560117)

In [118]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-c_pre'])
r, p

(-0.08193469802976337, 0.6248198925993775)

In [119]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-e_pre'])
r, p

(-0.06576519914503444, 0.6948465233078931)

In [120]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-a_pre'])
r, p

(0.09270929214477643, 0.5798511970052201)

In [121]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-n_pre'])
r, p

(0.23844009994505103, 0.14940647142893917)

In [122]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-o_post'])
r, p

(-0.008708007953332124, 0.9586183318186406)

In [123]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-c_post'])
r, p

(-0.03157563296161746, 0.8507277873337585)

In [124]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-e_post'])
r, p

(-0.08273878860676318, 0.6214145713298656)

In [125]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-a_post'])
r, p

(-0.0008203149458989804, 0.9961000886617819)

In [126]:
r, p = scipy.stats.pearsonr(dist_big5_valid['avg_dist'], dist_big5_valid['big5-n_post'])
r, p

(0.28393527316911915, 0.08406318087171535)

#### 1.7) travel distance + gpa
spring GPA & CS65 GPA가 significant

In [127]:
dist_gpa_valid = df[['uid', 'gpa_all', 'gpa_13s', 'cs_65', 'avg_dist']].dropna()

In [128]:
r, p = scipy.stats.pearsonr(dist_gpa_valid['avg_dist'], dist_gpa_valid['gpa_all'])
r, p

(-0.3118826617928208, 0.09338406039192966)

In [129]:
r, p = scipy.stats.pearsonr(dist_gpa_valid['avg_dist'], dist_gpa_valid['gpa_13s'])
r, p  # significant

(-0.5158971935134622, 0.0035217739063246375)

In [130]:
r, p = scipy.stats.pearsonr(dist_gpa_valid['avg_dist'], dist_gpa_valid['cs_65'])
r, p  #significant

(-0.5152207480811889, 0.0035727605170424765)

In [131]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_dist', 'gpa_13s', -0.516, 0.004]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

In [132]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_dist', 'cs_65', -0.515, 0.004]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

### 2) Number of locations visited
using DBSCAN

location detected when 3 GPS samples (1 sample every 10 minutes) are within a radius of 30 meters

#### visit same place twice -> counted as 2
--> 하루에 얼마나 돌아다니냐

In [133]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns

In [134]:
# for uid in gps_data:
#     for index in gps_data[uid].index:
        

# model = 

#### visit same place twice -> counted as 1
--> 얼마나 다양한 장소를 가냐

In [135]:
# modified from original source
# https://yidatao.github.io/2016-12-23/geolife-dbscan/

clusters = {}
for uid in gps_data:
    # represent GPS points as (lat, lon)
    coords = gps_data[uid][['latitude', 'longitude']].dropna()
    # earth's radius in km
    kms_per_radian = 6371.0088
    # define epsilon as 0.03 kilometers, converted to radians for use by haversine
    epsilon = 0.03 / kms_per_radian

    # eps is the max distance that points can be from each other to be considered in a cluster
    # min_samples is the minimum cluster size (everything else is classified as noise)
    db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    # get the number of clusters (ignore noisy samples which are given the label -1)
    num_clusters = len(set(cluster_labels) - set([-1]))

#     print ('Clustered ' + str(len(gps_data[uid])) + ' points to ' + str(num_clusters) + ' clusters')
    clusters[uid] = [uid, num_clusters]
    
locations = pd.DataFrame.from_dict(clusters, columns = ['uid', 'locations'], orient='index')


In [136]:
locations = locations.sort_values(by='uid').reset_index(drop=True)
locations.head()

,uid,locations
0,0,67
1,1,21
2,2,31
3,3,4
4,4,20


In [137]:
# merge
df = pd.merge(df, locations, how='outer').sort_values(by='uid').reset_index(drop=True)
df.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,...,panas-neg_pre,panas-pos_post,panas-neg_post,pss_pre,pss_post,gpa_all,gpa_13s,cs_65,avg_dist,locations
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,...,33.0,31.0,24.0,24.0,15.0,NaN,NaN,NaN,808.349,67
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,...,19.0,27.0,17.0,15.0,16.0,2.863,1.777778,3.0,1505.468,21
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,...,15.0,28.0,13.0,21.0,25.0,3.505,4.000000,4.0,852.401,31
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,...,27.0,25.0,20.0,17.0,24.0,NaN,NaN,NaN,486.413,4
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,...,14.0,34.0,19.0,21.0,17.0,3.029,3.500000,4.0,324.165,20


#### 2.1) locations + phq
post만 significant (r = -0.335, 0,040)

In [138]:
loc_phq_valid = df[['uid', 'locations', 'phq-9_pre', 'phq-9_post']].dropna()

In [139]:
r, p = scipy.stats.pearsonr(loc_phq_valid['locations'], loc_phq_valid['phq-9_pre'])
r, p

(-0.25032822668774446, 0.12956249983214757)

In [140]:
r, p = scipy.stats.pearsonr(loc_phq_valid['locations'], loc_phq_valid['phq-9_post'])
r, p  # significant

(-0.334646076248185, 0.04001281634296382)

In [141]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['locations', 'phq-9_post', -0.335, 0.040]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

#### 2.2) locations + flourishing
not significant

In [142]:
loc_fl_valid = df[['uid', 'locations', 'fl_pre', 'fl_post']].dropna()

In [143]:
r, p = scipy.stats.pearsonr(loc_fl_valid['locations'], loc_fl_valid['fl_pre'])
r, p

(0.254445484358243, 0.1464489692646061)

In [144]:
r, p = scipy.stats.pearsonr(loc_fl_valid['locations'], loc_fl_valid['fl_post'])
r, p

(0.3080406392683191, 0.07633351680756804)

#### 2.3) locations + loneliness
not significant

In [145]:
loc_lon_valid = df[['uid', 'locations', 'lon_pre', 'lon_post']].dropna()

In [146]:
r, p = scipy.stats.pearsonr(loc_lon_valid['locations'], loc_lon_valid['lon_pre'])
r, p

(-0.29426610440446854, 0.07707880519849686)

In [147]:
r, p = scipy.stats.pearsonr(loc_lon_valid['locations'], loc_lon_valid['lon_post'])
r, p

(-0.16090761509865928, 0.34140738420296435)

#### 2.4) locations + PANAS
not significant

In [148]:
loc_panas_valid = df[['uid', 'locations', 'panas-pos_pre', 'panas-neg_pre', 'panas-pos_post', 'panas-neg_post']].dropna()

In [149]:
r, p = scipy.stats.pearsonr(loc_panas_valid['locations'], loc_panas_valid['panas-pos_pre'])
r, p

(0.1027154561783781, 0.5394428655855347)

In [150]:
r, p = scipy.stats.pearsonr(loc_panas_valid['locations'], loc_panas_valid['panas-neg_pre'])
r, p

(-0.024092064677230133, 0.8858376282836163)

In [151]:
r, p = scipy.stats.pearsonr(loc_panas_valid['locations'], loc_panas_valid['panas-pos_post'])
r, p

(0.276483802216785, 0.09288713541084334)

In [152]:
r, p = scipy.stats.pearsonr(loc_panas_valid['locations'], loc_panas_valid['panas-neg_post'])
r, p

(-0.11225569712077363, 0.5022203156137869)

#### 2.5) locations + stress
not significant

In [153]:
loc_pss_valid = df[['uid', 'locations', 'pss_pre', 'pss_post']].dropna()

In [154]:
r, p = scipy.stats.pearsonr(loc_pss_valid['locations'], loc_pss_valid['pss_pre'])
r, p

(-0.08785916335640856, 0.5948235871965146)

In [155]:
r, p = scipy.stats.pearsonr(loc_pss_valid['locations'], loc_pss_valid['pss_post'])
r, p

(-0.16354798638952533, 0.3198113097104562)

#### 2.6) locations + big five
not significant

In [156]:
loc_big5_valid = df[['uid', 'locations', 'big5-o_pre', 'big5-c_pre', 
                              'big5-e_pre', 'big5-a_pre', 
                              'big5-n_pre', 'big5-o_post', 
                              'big5-c_post', 'big5-e_post', 
                              'big5-a_post', 'big5-n_post']].dropna()

In [157]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-o_pre'])
r, p

(0.31488680264927704, 0.054160234856770025)

In [158]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-c_pre'])
r, p

(-0.00019497764057198975, 0.999073041091481)

In [159]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-e_pre'])
r, p

(0.23307864087857708, 0.15904839959244754)

In [160]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-a_pre'])
r, p

(0.11625322417296674, 0.4870201447259423)

In [161]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-n_pre'])
r, p

(-0.2249082095605996, 0.17459674048133358)

In [162]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-o_post'])
r, p

(0.2505784580517133, 0.1291670036119806)

In [163]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-c_post'])
r, p

(0.001415219073398822, 0.993271867903872)

In [164]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-e_post'])
r, p

(0.19057505560315338, 0.25176147851234426)

In [165]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-a_post'])
r, p

(0.2793623878692929, 0.08939763586018983)

In [166]:
r, p = scipy.stats.pearsonr(loc_big5_valid['locations'], loc_big5_valid['big5-n_post'])
r, p

(-0.26723707789386303, 0.10480700162458116)

#### 2.7) locations + GPA
not significant

In [167]:
loc_gpa_valid = df[['uid', 'locations', 'gpa_all', 'gpa_13s', 'cs_65']].dropna()

In [168]:
r, p = scipy.stats.pearsonr(loc_gpa_valid['locations'], loc_gpa_valid['gpa_all'])
r, p

(0.08163145618035367, 0.6680452852078987)

In [169]:
r, p = scipy.stats.pearsonr(loc_gpa_valid['locations'], loc_gpa_valid['gpa_13s'])
r, p

(-0.11462297801223567, 0.5464206660162654)

In [170]:
r, p = scipy.stats.pearsonr(loc_gpa_valid['locations'], loc_gpa_valid['cs_65'])
r, p

(-0.0405314924603681, 0.8315967430212979)

### 3) Semantic Location

## 3. App Usage

In [171]:
# import app usage data
path = '//data/dopameter/student_life/dataset/dataset/app_usage/*.csv'
app_data = {}
for file_name in glob.glob(path):
    f = open(file_name, 'r')
    uid = int(re.search(r'\d+', file_name).group())  # user id
    app_data[uid] = pd.read_csv(f)  # dict {uid: light dataframe}

### 1) # of running activities

In [172]:
user_running = {}  # {uid: num running/activity df}
running_info = {}  # {uid: mean num running/activity}
# get total num activities, num running for each day
for key, value in app_data.items():
    curr_timestamp = None
    num_act_run = {}
    
    for i in value.index:
        row_num_activities = value.at[i, 'RUNNING_TASKS_numActivities']
        row_num_running = value.at[i, 'RUNNING_TASKS_numRunning']
        row_timestamp = value.at[i, 'timestamp']
        
        if row_timestamp == curr_timestamp:
            num_act_run[row_timestamp][0] += row_num_activities
            num_act_run[row_timestamp][1] += row_num_running
        else:
            curr_timestamp = row_timestamp
            num_act_run[row_timestamp] = [row_num_activities, row_num_running]

    running_df = pd.DataFrame.from_dict(num_act_run, columns = ['num_activities', 'num_running'], orient='index')
    
    user_running[key] = running_df
    
    # get mean num activities, num running per day
    running_info[key] = [round(running_df['num_activities'].mean(), 3), round(running_df['num_running'].mean(), 3)]

running = pd.DataFrame.from_dict(running_info, columns = ['avg_num_activities', 'avg_num_running'], orient='index')


In [173]:
running = running.reset_index().rename(columns={'index': 'uid'})
running = running.sort_values(by='uid').reset_index(drop=True)

In [174]:
# merge
df = pd.merge(df, running, how='outer').sort_values(by='uid').reset_index(drop=True)
df.head()

,uid,avg_bt,phq-9_pre,phq-9_post,phq-9_avg,fl_pre,fl_post,big5-o_pre,big5-c_pre,big5-e_pre,...,panas-neg_post,pss_pre,pss_post,gpa_all,gpa_13s,cs_65,avg_dist,locations,avg_num_activities,avg_num_running
0,0,3.738,2.0,3.0,2.5,NaN,45.0,45.0,27.0,30.0,...,24.0,24.0,15.0,NaN,NaN,NaN,808.349,67,14.426,11.397
1,1,1.851,5.0,4.0,4.5,45.0,46.0,39.0,40.0,25.0,...,17.0,15.0,16.0,2.863,1.777778,3.0,1505.468,21,9.082,8.044
2,2,2.653,13.0,5.0,9.0,46.0,44.0,32.0,38.0,17.0,...,13.0,21.0,25.0,3.505,4.000000,4.0,852.401,31,14.952,11.330
3,3,1.011,2.0,4.0,3.0,34.0,31.0,27.0,35.0,17.0,...,20.0,17.0,24.0,NaN,NaN,NaN,486.413,4,9.108,8.461
4,4,1.714,6.0,8.0,7.0,27.0,33.0,29.0,24.0,19.0,...,19.0,21.0,17.0,3.029,3.500000,4.0,324.165,20,10.487,7.742


#### 1.1) # running + phq
activities-pre, activities-post, running-pre가 significant

(r=0.623, p<.001) (r=0.459, p=0.004) (r=0.365, p=0.024)

In [175]:
run_phq_valid = df[['uid', 'avg_num_activities', 'avg_num_running', 'phq-9_pre', 'phq-9_post']].dropna()

In [176]:
r, p = scipy.stats.pearsonr(run_phq_valid['avg_num_activities'], run_phq_valid['phq-9_pre'])
r, p  # significant

(0.6226040613269249, 2.993547650148479e-05)

In [177]:
r, p = scipy.stats.pearsonr(run_phq_valid['avg_num_activities'], run_phq_valid['phq-9_post'])
r, p  # significant

(0.45859893012820496, 0.003782550520628265)

In [178]:
r, p = scipy.stats.pearsonr(run_phq_valid['avg_num_running'], run_phq_valid['phq-9_pre'])
r, p  # significant

(0.3650705939105718, 0.024215813074969807)

In [179]:
r, p = scipy.stats.pearsonr(run_phq_valid['avg_num_running'], run_phq_valid['phq-9_post'])
r, p

(0.1360127940556198, 0.4155160640324699)

In [180]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_num_activities', 'phq-9_pre', 0.623, 0.000]]
sig_arr += [['avg_num_activities', 'phq-9_post', 0.459, 0.004]]
sig_arr += [['avg_num_running', 'phq-9_pre', 0.459, 0.004]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

#### 1.2) # running + flourishing
not significant

In [181]:
run_fl_valid = df[['uid', 'avg_num_activities', 'avg_num_running', 'fl_pre', 'fl_post']].dropna()

In [182]:
r, p = scipy.stats.pearsonr(run_fl_valid['avg_num_activities'], run_fl_valid['fl_pre'])
r, p

(-0.2804798904449437, 0.10811337169356515)

In [183]:
r, p = scipy.stats.pearsonr(run_fl_valid['avg_num_activities'], run_fl_valid['fl_post'])
r, p

(-0.12893708516517208, 0.4673812952049468)

In [184]:
r, p = scipy.stats.pearsonr(run_fl_valid['avg_num_running'], run_fl_valid['fl_pre'])
r, p

(-0.1623502286136546, 0.3589572767811694)

In [185]:
r, p = scipy.stats.pearsonr(run_fl_valid['avg_num_running'], run_fl_valid['fl_post'])
r, p

(0.08031596394120599, 0.6516061105704756)

#### 1.3) # runing + loneliness
activities-post만 significant (r=0.350, p=0.034)

In [186]:
run_lon_valid = df[['uid', 'avg_num_activities', 'avg_num_running', 'lon_pre', 'lon_post']].dropna()

In [187]:
r, p = scipy.stats.pearsonr(run_lon_valid['avg_num_activities'], run_lon_valid['lon_pre'])
r, p

(0.20731154123149764, 0.21825782791828244)

In [188]:
r, p = scipy.stats.pearsonr(run_lon_valid['avg_num_activities'], run_lon_valid['lon_post'])
r, p  # significant

(0.34999034698118264, 0.03371062718933465)

In [189]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_num_activities', 'lon_post', 0.350, 0.034]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

In [190]:
r, p = scipy.stats.pearsonr(run_lon_valid['avg_num_running'], run_lon_valid['lon_pre'])
r, p

(0.1094312517237718, 0.5190966758306733)

In [191]:
r, p = scipy.stats.pearsonr(run_lon_valid['avg_num_running'], run_lon_valid['lon_post'])
r, p

(0.22924312438403768, 0.17230739406323736)

#### 1.4) # running + PANAS
not significant

In [192]:
run_panas_valid = df[['uid', 'avg_num_activities', 'avg_num_running', 'panas-pos_pre', 
                      'panas-neg_pre', 'panas-pos_post', 'panas-neg_post']].dropna()

In [193]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_activities'], run_panas_valid['panas-pos_pre'])
r, p

(-0.26427290353936267, 0.10886422498885646)

In [194]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_activities'], run_panas_valid['panas-neg_pre'])
r, p

(0.22830192361295737, 0.16801167970233075)

In [195]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_activities'], run_panas_valid['panas-pos_post'])
r, p

(-0.1007938239553216, 0.5470971097621141)

In [196]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_activities'], run_panas_valid['panas-neg_post'])
r, p

(0.29032204735623374, 0.07702967954531244)

In [197]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_running'], run_panas_valid['panas-pos_pre'])
r, p

(-0.12856348256046907, 0.441747938299362)

In [198]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_running'], run_panas_valid['panas-neg_pre'])
r, p

(0.046154654698376055, 0.7831943156508508)

In [199]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_running'], run_panas_valid['panas-pos_post'])
r, p

(-0.11800541276067711, 0.48043349132435437)

In [200]:
r, p = scipy.stats.pearsonr(run_panas_valid['avg_num_running'], run_panas_valid['panas-neg_post'])
r, p

(0.2554907832702279, 0.12158363822419578)

#### 1.5) # running + stress
activities-pre, activities-post가 significant

(r=0.416, p=0.008) (r=0.366, p=0.022)

In [201]:
run_pss_valid = df[['uid', 'avg_num_activities', 'avg_num_running',
                    'pss_pre', 'pss_post']].dropna()

In [202]:
r, p = scipy.stats.pearsonr(run_pss_valid['avg_num_activities'], run_pss_valid['pss_pre'])
r, p  # significant

(0.41590845862718023, 0.008455065333554857)

In [203]:
r, p = scipy.stats.pearsonr(run_pss_valid['avg_num_activities'], run_pss_valid['pss_post'])
r, p  # significant

(0.3663369009430329, 0.021806331355711053)

In [204]:
r, p = scipy.stats.pearsonr(run_pss_valid['avg_num_running'], run_pss_valid['pss_pre'])
r, p

(0.2907412826859032, 0.07255536580900547)

In [205]:
r, p = scipy.stats.pearsonr(run_pss_valid['avg_num_running'], run_pss_valid['pss_post'])
r, p

(0.28255334266801, 0.08136055579072544)

In [206]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['avg_num_activities', 'pss_pre', 0.416, 0.008]]
sig_arr += [['avg_num_activities', 'pss_post', 0.366, 0.022]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr = sig_corr.reset_index(drop=True)
sig_corr.to_csv('sig_corr.csv')

#### 1.6) # running + big five
not significant

In [207]:
run_big5_valid = df[['uid', 'avg_num_activities', 'avg_num_running', 
                              'big5-o_pre', 'big5-c_pre', 
                              'big5-e_pre', 'big5-a_pre', 
                              'big5-n_pre', 'big5-o_post', 
                              'big5-c_post', 'big5-e_post', 
                              'big5-a_post', 'big5-n_post']].dropna()

In [208]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-o_pre'])
r, p

(0.1778656117393432, 0.2853589579826848)

In [209]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-c_pre'])
r, p

(-0.276199038172959, 0.09323794450752114)

In [210]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-e_pre'])
r, p

(0.024189670345790348, 0.8853781691739276)

In [211]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-a_pre'])
r, p

(-0.2120516465735584, 0.2012137888240153)

In [212]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-n_pre'])
r, p

(0.16406671508509277, 0.3249774503029454)

In [213]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-o_post'])
r, p

(0.1945855933257481, 0.2417324418231729)

In [214]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-c_post'])
r, p

(-0.2502844309676262, 0.12963181241750638)

In [215]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-e_post'])
r, p

(0.05129458289616867, 0.7597266973238297)

In [216]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-a_post'])
r, p

(-0.07503475602090087, 0.6543509299817675)

In [217]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_activities'], run_big5_valid['big5-n_post'])
r, p

(0.1961872329127654, 0.23780357312278302)

In [218]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-o_pre'])
r, p

(-0.06613329634069862, 0.6932215088249785)

In [219]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-c_pre'])
r, p

(0.06506173641568767, 0.6979557866511938)

In [220]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-e_pre'])
r, p

(0.009079154006769431, 0.9568562075194302)

In [221]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-a_pre'])
r, p

(-0.2857482858965752, 0.08201786218609364)

In [222]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-n_pre'])
r, p

(0.06485296324633928, 0.6988794894057411)

In [223]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-o_post'])
r, p

(0.015488816063965079, 0.9264633462622406)

In [224]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-c_post'])
r, p

(0.013092759531952986, 0.937815058039863)

In [225]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-e_post'])
r, p

(-0.0976490301358509, 0.5597333741507309)

In [226]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-a_post'])
r, p

(-0.19166130616597973, 0.2490181064490336)

In [227]:
r, p = scipy.stats.pearsonr(run_big5_valid['avg_num_running'], run_big5_valid['big5-n_post'])
r, p

(0.14179656483912037, 0.39577205345136346)

#### 1.7) # running + GPA
not significant

In [228]:
run_gpa_valid = df[['uid', 'avg_num_activities', 'avg_num_running', 
                    'gpa_all', 'gpa_13s', 'cs_65']].dropna()

In [229]:
r, p = scipy.stats.pearsonr(run_gpa_valid['avg_num_activities'], run_gpa_valid['gpa_all'])
r, p

(-0.028345170015547864, 0.8818017534312977)

In [230]:
r, p = scipy.stats.pearsonr(run_gpa_valid['avg_num_activities'], run_gpa_valid['gpa_13s'])
r, p

(-0.06948445185802728, 0.7152219077455737)

In [231]:
r, p = scipy.stats.pearsonr(run_gpa_valid['avg_num_activities'], run_gpa_valid['cs_65'])
r, p

(0.18218068165716264, 0.335276304894662)

In [232]:
r, p = scipy.stats.pearsonr(run_gpa_valid['avg_num_running'], run_gpa_valid['gpa_all'])
r, p

(0.32843516039862425, 0.07639408287793785)

In [233]:
r, p = scipy.stats.pearsonr(run_gpa_valid['avg_num_running'], run_gpa_valid['gpa_13s'])
r, p

(0.1264739764553849, 0.5054279111147543)

In [234]:
r, p = scipy.stats.pearsonr(run_gpa_valid['avg_num_running'], run_gpa_valid['cs_65'])
r, p

(0.2674741949205507, 0.15301756694789886)

In [235]:
sig_corr

,sig1,sig2,r,p
0,avg_bt,phq-9_post,-0.363,0.025
1,avg_bt,big5-a_pre,-0.324,0.047
2,avg_bt,gpa_13s,0.447,0.048
3,avg_dist,gpa_13s,-0.516,0.004
4,avg_dist,cs_65,-0.515,0.004
5,locations,phq-9_post,-0.335,0.040
6,avg_num_activities,phq-9_pre,0.623,0.000
7,avg_num_activities,phq-9_post,0.459,0.004
8,avg_num_running,phq-9_pre,0.459,0.004
9,avg_num_activities,lon_post,0.350,0.034


#### correlations summary
|Feature|Scale|r|p|
|:---|:---|---:|--:|
|**average # of bluetooth devices**|depression|-0.363|0.025|
||agreeableness|-0.324|0.047|
||GPA (spring term)|0.447|0.048|
|**average traveled distance**|GPA (spring term)|-0.516|0.004|
|**# of location clusters**|depression|-0.335|0.040|
|**# of running applications**|depression|0.623|0.001|
||loneliness|0.350|0.034|
||stress|0.416|0.008|


### 2) App categories

In [237]:
import requests
from bs4 import BeautifulSoup

def get_html(url):
    html = ""
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
    return html

GOOGLE_URL = "https://play.google.com/store/apps/details?id=";

# social = ['Communication', 'Social', 'Dating']
# engagement = ['Tools', 'Travel & Local', 'Productivity', 'Food & Drink', 'Libraries & Demo',
#               'Health & Fitness', 'Finance', 'Personalization', 'Weather', 'Lifestyle',
#               'Casual', 'Board', 'Strategy', 'Sports', 'Business', 'Maps & Navigation']
# entertainment = ['Video Players & Editors', 'News & Magazines', 'Education', 
#                  'Music & Audio', 'Photography', 'Entertainment', 'Books & Reference',
#                  'Puzzle', 'Arcade', 'Shopping', 'Word', 'Simulation', 'Action', 'Racing',
#                  'Adventure', 'Role Playing', 'Card']
# user_cat_nums = {}  # {uid: df of time & number of apps in each category}
# user_cat_increase = {}  # {uid: df of time & increase in number of apps in each category}
# categories = {}  # save categories of packages in order not to send requests too many times
# # get total num activities, num running for each day
# for key, value in app_data.items():
#     curr_timestamp = None
#     cat_nums = {}
    
#     for i in value.index:
#         row_topActivity = value.at[i, 'RUNNING_TASKS_topActivity_mPackage']
#         row_timestamp = value.at[i, 'timestamp']

#         category = ''
#         cat = -1
#         if row_topActivity in categories:
#             category = categories[row_topActivity]
#         else:
#             html = get_html(GOOGLE_URL + row_topActivity)
#             soup = BeautifulSoup(html, 'html.parser')
#             tags = soup.findAll('a', attrs={'itemprop': 'genre'})

#             if tags:
#                 for tag in tags:  # must be only one tag in tags
#                     categories[row_topActivity] = tag.text
#                     category = tag.text
#             else:
#                 categories[row_topActivity] = ''  # if no category in playstore, save as '' so it 
#                                                   # doesn't look for category anymore

#         if category in social:
#             cat = 0  # social
#         elif category in engagement:
#             cat = 1  # engagement
#         elif category in entertainment:
#             cat = 2  # entertainment
#         elif category == '':
#             pass
#         else:
#             print(category)
#             print(row_topActivity)

#         if row_timestamp == curr_timestamp:
#             if cat == 0:
#                 cat_nums[row_timestamp][0] += 1
#             elif cat == 1:
#                 cat_nums[row_timestamp][1] += 1
#             elif cat == 2:
#                 cat_nums[row_timestamp][2] += 1
#         else:
#             if cat == 0:
#                 cat_nums[row_timestamp] = [row_timestamp, 1, 0, 0]
#             elif cat == 1:
#                 cat_nums[row_timestamp] = [row_timestamp, 0, 1, 0]
#             elif cat == 2:
#                 cat_nums[row_timestamp] = [row_timestamp, 0, 0, 1]
#             elif cat == -1:
#                 cat_nums[row_timestamp] = [row_timestamp, 0, 0, 0]
#         curr_timestamp = row_timestamp

#     cat_nums_df = pd.DataFrame.from_dict(cat_nums, columns = ['timestamp', 'social', 
#                                                               'engagement', 'entertainment'], 
#                                          orient='index').sort_values(by='timestamp') \
#                                                         .reset_index(drop=True)
#     user_cat_nums[key] = cat_nums_df

#     # get increase of each category
#     cat_increase_df = cat_nums_df.copy()
#     first = True
#     for i in cat_increase_df.index:
#         if first:
#             first = False
#             cat_increase_df.loc[i] = [0, 0, 0, 0]
#         else:
#             term = cat_nums_df.iat[i, 0] - cat_nums_df.iat[i-1, 0]
#             soc = cat_nums_df.iat[i, 1] - cat_nums_df.iat[i-1, 1]
#             eng = cat_nums_df.iat[i, 2] - cat_nums_df.iat[i-1, 2]
#             ent = cat_nums_df.iat[i, 3] - cat_nums_df.iat[i-1, 3]
#             cat_increase_df.iat[i, 0] = term
#             cat_increase_df.iat[i, 1] = max(soc, 0) * 1000000 / term  # divide by time interval
#             cat_increase_df.iat[i, 2] = max(eng, 0) * 1000000 / term
#             cat_increase_df.iat[i, 3] = max(ent, 0) * 1000000 / term
            
#     user_cat_increase[key] = cat_increase_df

In [238]:
# get app name from package name
app_names = {}  # save app names of packages in order not to send requests too many times
for key, value in app_data.items():
    
#     for i in value.index:
    row_topActivity = value.at[0, 'RUNNING_TASKS_topActivity_mPackage']
    row_timestamp = value.at[0, 'timestamp']

    app_name = ''

    if row_topActivity in app_names:
        app_name = app_names[row_topActivity]
    else:
        html = get_html(GOOGLE_URL + row_topActivity)
        soup = BeautifulSoup(html, 'html.parser')
        tags = soup.findAll('h1', attrs={'itemprop': 'name'})

        if tags:
            for tag in tags:  # must be only one tag in tags
                print(tag.text)
                app_names[row_topActivity] = tag.text
                app_name = tag.text
        else:
            app_names[row_topActivity] = ''  # if no category in playstore, save as '' so it 
                                              # doesn't look for category anymore

Google
Gmail
Hangouts
Samsung One UI Home


In [ ]:
# get mean of increased apps for each category
app_cat = []
for key, value in user_cat_increase.items():
    app_cat += [[key, round(value['social'].mean(), 3), round(value['engagement'].mean(), 3), 
                 round(value['entertainment'].mean(), 3)]]
app_cat_df = pd.DataFrame(app_cat, columns=['uid', 'social', 'engagement', 'entertainment'])

In [ ]:
app_cat_df

In [ ]:
app_cat_df = app_cat_df.sort_values(by='uid').reset_index(drop=True)
app_cat_df.head()

In [ ]:
# merge
df = pd.merge(df, app_cat_df, how='outer').sort_values(by='uid').reset_index(drop=True)
df.head()

#### 2.1) app category + phq
pre, entertainment만 significant (r=0.363, p=0.025)

In [ ]:
app_phq_valid = df[['uid', 'social', 'engagement', 
                    'entertainment', 'phq-9_pre', 'phq-9_post']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(app_phq_valid['social'], app_phq_valid['phq-9_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(app_phq_valid['engagement'], app_phq_valid['phq-9_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(app_phq_valid['entertainment'], app_phq_valid['phq-9_pre'])
r, p  # significant

In [ ]:
r, p = scipy.stats.pearsonr(app_phq_valid['social'], app_phq_valid['phq-9_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(app_phq_valid['engagement'], app_phq_valid['phq-9_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(app_phq_valid['entertainment'], app_phq_valid['phq-9_post'])
r, p

In [ ]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['entertainment', 'phq-9_pre', 0.363, 0.025]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

#### 2.2) app category + flourishing
entertainment, pre만 significant (r=-0.341, p=0.048)

In [ ]:
valid_df = df[['uid', 'social', 'engagement', 
                    'entertainment', 'fl_pre', 'fl_post']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['fl_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['fl_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['fl_pre'])
r, p  # significant

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['fl_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['fl_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['fl_post'])
r, p

In [ ]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['entertainment', 'fl_pre', -0.341, 0.048]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

#### 2.3) app category + loneliness
not significant

In [ ]:
valid_df = df[['uid', 'social', 'engagement', 
                    'entertainment', 'lon_pre', 'lon_post']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['lon_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['lon_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['lon_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['lon_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['lon_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['lon_post'])
r, p

#### 2.4) app category + PANAS
entertainment, positive (pre)만 significant (r=0.201, p=0.227)

In [ ]:
valid_df = df[['uid', 'social', 'engagement', 
               'entertainment', 'panas-pos_pre', 'panas-pos_post', 
               'panas-neg_pre', 'panas-neg_post']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['panas-pos_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['panas-pos_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['panas-pos_pre'])
r, p  # significant

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['panas-pos_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['panas-pos_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['panas-pos_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['panas-neg_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['panas-neg_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['panas-neg_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['panas-neg_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['panas-neg_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['panas-neg_post'])
r, p

In [ ]:
# save significant associations in a dataframe (-> export as csv file)
sig_arr = [['entertainment', 'panas-pos_pre', 0.201, 0.227]]
sig_corr = sig_corr.append(pd.DataFrame(sig_arr, columns=['sig1', 'sig2', 'r', 'p']))
sig_corr.to_csv('sig_corr.csv')

#### 2.5) app category + Big Five
not significant

In [ ]:
valid_df = df[['uid', 'social', 'engagement', 
               'entertainment', 'big5-o_pre', 'big5-o_post', 'big5-c_pre', 'big5-c_post',
               'big5-e_pre', 'big5-e_post', 'big5-a_pre', 'big5-a_post', 'big5-n_pre',
               'big5-n_post']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-o_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-o_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-o_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-o_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-o_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-o_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-c_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-c_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-c_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-c_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-c_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-c_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-e_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-e_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-e_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-e_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-e_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-e_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-a_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-a_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-a_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-a_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-a_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-a_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-n_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-n_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-n_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['big5-n_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['big5-n_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['big5-n_post'])
r, p

#### 2.6) app category + stress
not significant

In [ ]:
valid_df = df[['uid', 'social', 'engagement', 
               'entertainment', 'pss_pre', 'pss_post']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['pss_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['pss_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['pss_pre'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['pss_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['pss_post'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['pss_post'])
r, p

#### 2.7) app category + GPA
not significant

In [ ]:
valid_df = df[['uid', 'social', 'engagement', 
               'entertainment', 'gpa_all', 'gpa_13s', 'cs_65']].dropna()

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['gpa_all'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['gpa_all'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['gpa_all'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['gpa_13s'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['gpa_13s'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['gpa_13s'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['social'], valid_df['cs_65'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['engagement'], valid_df['cs_65'])
r, p

In [ ]:
r, p = scipy.stats.pearsonr(valid_df['entertainment'], valid_df['cs_65'])
r, p

## Light


In [ ]:
# import light data
path = '//data/dopameter/student_life/dataset/dataset/sensing/dark/*.csv'
light_data = {}
for file_name in glob.glob(bt_path):
    f = open(file_name, 'r')
    uid = int(re.search(r'\d+', file_name).group())  # user id
    light_data[uid] = pd.read_csv(f)  # dict {uid: light dataframe}

In [ ]:
df.to_csv('user_data.csv')